In [ ]:
import numpy as np
import qubex as qx
from qubex.simulator import Control, Coupling, QuantumSimulator, QuantumSystem, Transmon

In [ ]:
qubits = [
    Transmon(
        label="Q01",
        dimension=3,
        frequency=7.648,
        anharmonicity=-0.33,
        relaxation_rate=0.00005,
        dephasing_rate=0.00005,
    ),
    Transmon(
        label="Q02",
        dimension=3,
        frequency=8.275,
        anharmonicity=-0.33,
        relaxation_rate=0.00005,
        dephasing_rate=0.00005,
    ),
]

system = QuantumSystem(
    objects=qubits,
    couplings=[
        Coupling(
            pair=("Q01", "Q02"),
            strength=0.01,
        ),
    ],
)

simulator = QuantumSimulator(system)

In [ ]:
system.hamiltonian

In [ ]:
duration = 30
tau = 10

x180 = qx.pulse.FlatTop(
    duration=duration,
    amplitude=np.pi / (duration - tau),
    tau=tau,
)
y180 = x180.shifted(np.pi / 2)

with qx.PulseSchedule() as ps:
    ps.add("Q01", x180)
    ps.barrier()
    ps.add("Q01", y180)
    ps.add("Q02", x180)
    ps.add("Q02", y180)

ps.plot()

waveforms = ps.get_sampled_sequences()

In [ ]:
control_0 = Control(
    target=qubits[0].label,
    frequency=qubits[0].frequency,
    waveform=waveforms[qubits[0].label],
)
control_0.plot()

control_1 = Control(
    target=qubits[1].label,
    frequency=qubits[1].frequency,
    waveform=waveforms[qubits[1].label],
)
control_1.plot()

In [ ]:
initial_state = system.state({
    "Q01": "0",
    "Q02": "1",
})

initial_state

In [ ]:
result = simulator.mesolve(
    controls=[control_0, control_1],
    initial_state=initial_state,
)

In [ ]:
final_state = result.states[-1]
final_state

In [ ]:
result.plot_population_dynamics()
result.plot_population_dynamics(qubits[0].label)
result.plot_population_dynamics(qubits[1].label)

In [ ]:
result.display_bloch_sphere(qubits[0].label)
result.display_bloch_sphere(qubits[1].label)